<a href="https://colab.research.google.com/github/pjluvi/RAG_with_chat_history/blob/main/RAG_with_chathistory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 21.4 MB/s eta 0:

In [2]:
from google.colab import userdata
#OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [3]:
import os
#os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [4]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [5]:
!pip install -U langchain_google_genai

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [7]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

In [9]:
llm.invoke("what is a value of 2+3?").content

'The value of 2 + 3 is **5**. \n'

In [10]:
from langchain_chroma import Chroma

In [11]:
from langchain_community.document_loaders import WebBaseLoader

In [12]:
from langchain_core.output_parsers import StrOutputParser

In [13]:
from langchain_core.runnables import RunnablePassthrough

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [15]:
import bs4

In [16]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

In [17]:
docs=loader.load()

In [18]:
print(docs[0].page_content)



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Short-term memory: I 

In [19]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [20]:
splits =text_splitter.split_documents(docs)

In [21]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 6.3 MB/s eta 0:00:00


In [22]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [23]:
model_name = "BAAI/bge-small-en-v1.5"

In [24]:
encode_kwargs = {'normalize_embeddings': True}

In [25]:
embedding_function=HuggingFaceBgeEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [26]:
vectorstore = Chroma.from_documents(splits,embedding_function)

In [27]:
retriever=vectorstore.as_retriever()

In [28]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7f16f3d7b3d0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x7f16f3daebf0>, default_metadata=())

In [29]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [30]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:322: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langsmith/client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [31]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [32]:
rag_chain=(
    {"context" : retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [34]:
rag_chain.invoke("can you give me a Types of Memory?")

'The main types of memory are sensory memory, short-term memory (STM), and long-term memory (LTM). Sensory memory briefly holds sensory information, STM stores information actively being used, and LTM stores information for long periods, including both explicit (consciously recalled) and implicit (unconscious, like skills) memories. \n'

In [35]:
from langchain.chains import create_history_aware_retriever

In [36]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [37]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

In [38]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [39]:
history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
    contextualize_q_prompt,
)

In [40]:
from langchain.chains import create_retrieval_chain

In [41]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [42]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""





In [43]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [44]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)


In [45]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [46]:
from langchain_core.messages import HumanMessage

In [47]:
chat_history=[]

In [48]:
question1= "What is Task Decomposition?"

In [49]:
response1=rag_chain.invoke({"input":question1,"chat_history": chat_history})

In [50]:
chat_history.extend([HumanMessage(content=question1), response1["answer"]])

In [51]:
print(response1['answer'])

Task Decomposition in AI refers to breaking down a complex task into smaller, more manageable subtasks. This approach, often used in Chain of Thought prompting, helps the model understand and address the problem step-by-step. As a result, it improves the model's performance on complex tasks and provides insights into its reasoning process. 



Task Decomposition is the process of breaking down a complex task into smaller, more manageable subtasks. This approach, often used in Chain-of-Thought prompting, helps LLMs understand and address complex problems by tackling them step-by-step. The decomposition can be facilitated by the LLM itself, guided by specific instructions, or provided by human input.



In [52]:
response1

{'input': 'What is Task Decomposition?',
 'chat_history': [HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}),
  "Task Decomposition in AI refers to breaking down a complex task into smaller, more manageable subtasks. This approach, often used in Chain of Thought prompting, helps the model understand and address the problem step-by-step. As a result, it improves the model's performance on complex tasks and provides insights into its reasoning process. \n"],
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or maj

In [53]:
question2="what are common ways of doing it?"

In [54]:
response2=rag_chain.invoke({"input":question2,"chat_history": chat_history})

In [55]:
response2

{'input': 'what are common ways of doing it?',
 'chat_history': [HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}),
  "Task Decomposition in AI refers to breaking down a complex task into smaller, more manageable subtasks. This approach, often used in Chain of Thought prompting, helps the model understand and address the problem step-by-step. As a result, it improves the model's performance on complex tasks and provides insights into its reasoning process. \n"],
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think

In [56]:
response2["answer"]

'Task decomposition can be done by: (1) prompting LLMs with simple instructions to break down the task; (2) using task-specific instructions, like providing a story outline for writing a novel; or (3) directly involving humans in the decomposition process. \n'

Task decomposition can be achieved by prompting LLMs with simple instructions like "Steps for XYZ" or "Sub-goals for achieving XYZ." Alternatively, you can use task-specific instructions, like "Write a story outline" for writing a novel, or provide human input to guide the decomposition process.


In [57]:
response2

{'input': 'what are common ways of doing it?',
 'chat_history': [HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}),
  "Task Decomposition in AI refers to breaking down a complex task into smaller, more manageable subtasks. This approach, often used in Chain of Thought prompting, helps the model understand and address the problem step-by-step. As a result, it improves the model's performance on complex tasks and provides insights into its reasoning process. \n"],
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think

In [58]:
chat_history.extend([HumanMessage(content=question2), response2["answer"]])

In [59]:
question3="for writing a novel what i can do?"

In [60]:
response3=rag_chain.invoke({"input":question3,"chat_history": chat_history})

In [61]:
response3["answer"]

'For writing a novel with task decomposition, you can prompt an LLM with instructions like "Write a story outline," or  "Create a list of characters and their motivations." This will break down the complex task of novel writing into smaller, manageable steps. \n'

In [62]:
response3

{'input': 'for writing a novel what i can do?',
 'chat_history': [HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}),
  "Task Decomposition in AI refers to breaking down a complex task into smaller, more manageable subtasks. This approach, often used in Chain of Thought prompting, helps the model understand and address the problem step-by-step. As a result, it improves the model's performance on complex tasks and provides insights into its reasoning process. \n",
  HumanMessage(content='what are common ways of doing it?', additional_kwargs={}, response_metadata={}),
  'Task decomposition can be done by: (1) prompting LLMs with simple instructions to break down the task; (2) using task-specific instructions, like providing a story outline for writing a novel; or (3) directly involving humans in the decomposition process. \n'],
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Tree